In [1]:
import nltk

In [2]:
gram_lab = nltk.CFG.fromstring("""  S -> NP VP
                                    S -> VP
                                    NP -> DT NN
                                    NP -> DT JJ NN
                                    NP -> PRP
                                    VP -> VBP NP
                                    VP -> VBP VP
                                    VP -> VBG NP
                                    VP -> TO VP
                                    VP -> VB
                                    VP -> VB NP
                                    NN -> "show" | "book"
                                    PRP -> "I"
                                    VBP -> "am"
                                    VBG -> "watching"
                                    VB -> "show"
                                    DT -> "a" | "the"
                                    MD -> "will"  """)

In [3]:
def Bottom_Up(grammar, bottom, goal):
    stack = []
    result_ = []
    start = list(grammar.productions(rhs = bottom)) # Get the productions that have the starting word in rhs
    
    for prod in start: #direct match (show & NN case)
        if (prod.lhs() == goal):
            result_.append([prod])
        
    while len(start) > 0:   # As long as we did not reach the goal
        w = start[0]
        start.pop(0) # for each prod containing the starting word
        stack.append(w) # store that prod
        
        productions = grammar.productions(rhs = w.lhs()) # get next productions
        
        for prod in productions:
            if prod.lhs() == goal:  # check if they match the goal
                stack.append(prod)
                
                res = []
                for el in stack:
                    res.append(el)
                    
                result_.append(res)
                stack.pop(1)
            start.insert(0, prod)         # Get next prod
        
    return result_

In [4]:
Bottom_Up(gram_lab, 'am', nltk.Nonterminal('VP'))  # multiple cases

[[VBP -> 'am', VP -> VBP NP], [VBP -> 'am', VP -> VBP VP]]

In [5]:
Bottom_Up(gram_lab, 'watching', nltk.Nonterminal('S'))  # multiple prod

[[VBG -> 'watching', VP -> VBG NP, S -> VP]]

In [6]:
Bottom_Up(gram_lab, 'show', nltk.Nonterminal('NN'))  # direct match

[[NN -> 'show']]

In [7]:
Bottom_Up(gram_lab, 'a', nltk.Nonterminal('VP'))  # does not match because we use the left corner table

[]

In [8]:
sentence = 'I am wathing a show'
words = ['I', 'am', 'watching', 'a', 'show']

In [9]:
Bottom_Up(gram_lab, 'I', nltk.Nonterminal('S'))

[[PRP -> 'I', NP -> PRP, S -> NP VP]]

In [10]:
def left_corner_parser(grammar, data, goal, i = 0):
    
    
    productions = Bottom_Up(grammar, data[i], goal)
    if not productions:    # No match
        return False
    
    for production in productions: # Go up until we find the next goal
        for prod in production[1:]:
            if len(prod.rhs()) != 1: # Top down (S->NP VP)
                for element in prod.rhs()[1:]: # prod.rhs()[1] is where we came from (in the left corner table) (NP)
                                            # we want to perform top down for the next branch that we just found (VP)
                    if i + 1 >= len(data):        # if we have a new branch but no words to continue => Fail => backtrack
                        break
                    
                    re = left_corner_parser(grammar, data, element, i + 1)  # Match next word with the new banch (am & VP)
                    
                    if not re:               # if there is no match => Fail => backtrack
                        break
                        
    print(production)
    return True


In [11]:
left_corner_parser(gram_lab, ['I', 'am', 'watching', 'a', 'show'], gram_lab.start())

[NN -> 'show']
[DT -> 'a', NP -> DT JJ NN]
[VBG -> 'watching', VP -> VBG NP]
[VBP -> 'am', VP -> VBP VP]
[PRP -> 'I', NP -> PRP, S -> NP VP]


True

In [12]:
left_corner_parser(gram_lab, ['I', 'book', 'am', 'show'], gram_lab.start())

[PRP -> 'I', NP -> PRP, S -> NP VP]


True